In [1]:
import pandas as pd
import os
from IPython.core.display import HTML

In [2]:
def local_pdf_path(row):
    mapname = row["map"].removesuffix(".map")
    path = "./benchmarks/%s/%s.pdf" % (row["domain"], mapname)
    return path

def local_png_path(row):
    mapname = row["map"].removesuffix(".map")
    path = "../benchmarks/%s/%s.png" % (row["domain"], mapname)
    return path

def png_path(row):
    mapname = row["map"].removesuffix(".map")
    url = "https://movingai.com/benchmarks/%s/%s_small.png" % (row["domain"], mapname)
    return url

def pdf2html(path):
    return '<embed src="' + path + 'width="60px" height="60px"'

def png2html(path):
    return '<a href="%s" target="_blank"><img src="%s" width="100px"></a>' % (path, path)

def pnglist2html(pngs):
    return " ".join([png2html(i) for i in pngs])


In [3]:
df = pd.read_csv("meta-info.csv")
desc = pd.read_csv("labels.csv")
df = pd.merge(df, desc, on=["domain", "label"])
df["preview"] = df.apply(lambda x: local_png_path(x), axis=1)
df.shape

(659, 5)

In [4]:
# load meta info
files = os.listdir("domains")
dfs = []
for i in files:
    t = pd.read_csv("domains/"+i, sep="\t")
    t["domain"] = i.removesuffix(".csv")
    t["map"] = t["map"] + ".map"
    dfs.append(t)
metas = pd.concat(dfs)
metas.shape

(824, 16)

In [5]:
# add meta columns
t = pd.merge(df, metas, on=["map", "domain"])
# filter out similar maps
with open("../ignore.txt", "r") as f:
    ignores = [i.strip().split("/")[-1] for i in f.readlines()]
t = t[~t["map"].isin(ignores)].reset_index()
t.shape

(398, 20)

In [7]:
# filter by top-k gen-mean
topk = 5
idx = t.groupby(["domain", "label", "desc"])["gen-mean"].nlargest(topk).reset_index()["level_3"]
selected = t.iloc[idx].groupby(["domain", "label", "desc"]).apply(lambda x: x["preview"].to_list())
selected = selected.to_frame("previews").reset_index()
selected.to_html("selected.html", escape=False, formatters=dict(previews=pnglist2html))

In [8]:
# create selected map folder
selected_maps = t.iloc[idx][["map", "domain"]]
for domain in selected_maps["domain"].unique():
    os.system("mkdir -p ./selected/%s" % domain)
for idx, row in selected_maps.iterrows():
    domain, mapname = row["domain"], row["map"]
    os.system("cp ../benchmarks/%s/%s ./selected/%s/" % (domain, mapname, domain))

In [9]:
t = df.groupby(["domain", "label", "desc"]).apply(lambda x: x["preview"].to_list())
t = t.to_frame("previews").reset_index()
t.to_html("visual.html", escape=False, formatters=dict(previews=pnglist2html), columns=["domain", "label", "desc", "previews"])